# Отчаянные домохозяйки в СНТ "Вентилятор-2"

В 5 сезоне одного популярного сериала появилось HOA - HomeOwners Association (некая организация жильцов, которая может решать совместные проблемы, такие как вызов мусора, собирать деньги с жильцов на общие нужды и штрафовать за нарушения (violations) устава). В России такое называется Товарищество Собственников Недвижимости (Жилья), и наиболее популярный вид - Садоводческое Некоммерческое Товарищество (СНТ).

**Наша цель** - разработка кодовой основы под модуль, который реализовывал бы основной функционал HOA/СНТ :)

## Отчаянные домохозяйки - 1

Реализуй следующие классы: `address` (отвечает за непосредственно адреса с улицей, номером дома, корпусом (если есть)), `person` (отвечает за конкретного жителя - его фамилия, имя, отчество; они же в формате Иванов И.И.); `household` - класс, который отвечает за соотношение, какие жильцы где живут, и информацию о конкретных домах (сколько стоит дом, сколько было нарушений у жильцов этого дома, пр.)

Используй dataclass (возможно, с параметрами `frozen=True` или `slots=True`) и property там, где нужно.

In [50]:
from dataclasses import dataclass, field
from typing import List, Optional
from datetime import datetime

@dataclass(frozen=True, slots=True)
class Address:
    street: str
    house_number: str
    building: Optional[str] = None
    
    @property
    def full_address(self) -> str:
        address_parts = [
            f"ул. {self.street}",
            f"д. {self.house_number}"
        ]
        if self.building:
            address_parts.append(f"корп. {self.building}")
        return ", ".join(address_parts)
    
    def __str__(self) -> str:
        return self.full_address

@dataclass(slots=True)
class Person:
    last_name: str
    first_name: str
    middle_name: Optional[str] = None
    
    @property
    def short_name(self) -> str:
        first_initial = f"{self.first_name[0]}."
        middle_initial = f"{self.middle_name[0]}." if self.middle_name else ""
        return f"{self.last_name} {first_initial}{middle_initial}"
    
    def __str__(self) -> str:
        return self.short_name

@dataclass(slots=True)
class Violation:
    description: str
    date: datetime
    fine_amount: float
    is_paid: bool = False
    id: Optional[int] = None
    
    @property
    def status(self) -> str:
        if self.is_paid:
            return "Оплачено" 
        else:
            "Не оплачено"
    
    def mark_as_paid(self) -> None:
        self.is_paid = True

@dataclass(slots=True)
class Household:
    address: Address
    owner: Person
    residents: List[Person] = field(default_factory=list)
    violations: List[Violation] = field(default_factory=list)
    id: Optional[int] = None
    
    @property
    def total_fines(self) -> float:
        return sum(violation.fine_amount 
                  for violation in self.violations 
                  if not violation.is_paid)
    
    @property
    def violations_count(self) -> int:
        return len(self.violations)
    
    @property
    def unpaid_violations_count(self) -> int:
        return sum(1 for violation in self.violations if not violation.is_paid)
    
    @property
    def residents_count(self) -> int:
        return len(self.residents)

    @property
    def has_residents(self) -> bool:
        return len(self.residents) > 0
    
    def add_resident(self, person: Person) -> None:
        if person not in self.residents:
            self.residents.append(person)
    
    def add_violation(self, violation: Violation) -> None:
        self.violations.append(violation)
    
    def pay_fine(self, violation_id: int) -> bool:
        for violation in self.violations:
            if violation.id == violation_id and not violation.is_paid:
                violation.mark_as_paid()
                return True
        return False



In [53]:
person1 = Person("Иванов", "Иван", "Иванович")
person2 = Person("Петров", "Петр", "Петрович")
person3 = Person("Сидорова", "Мария", "Сергеевна")
print(person1, person2)


address1 = Address("Ленина", "15", "А")
address2 = Address("Пушкина", "42")
print(address1)
household1 = Household(
    address=address1,
    owner=person1,
    residents=[person1, person2],
)
      
household2 = Household(
    address=address2,
    owner=person3,
)

violation1 = Violation(
    description="Неуборка снега с тротуара",
    date=datetime.now(),
    fine_amount=1500.0
)

violation2 = Violation(
    description="Несанкционированная парковка",
    date=datetime.now(),
    fine_amount=2000.0
)
    
household1.add_violation(violation1)
household2.add_violation(violation2)

violation1.mark_as_paid()
print(violation1.status)

print(household2.total_fines)
print(household1.violations_count)
print(household1.unpaid_violations_count)
print(household1.residents_count)
household1.add_resident(person3)
print(household1.residents_count)
household1.add_violation(violation1)
print(household1.violations_count)
household1.pay_fine(2)
print(household1.unpaid_violations_count)

Иванов И.И. Петров П.П.
ул. Ленина, д. 15, корп. А
Оплачено
2000.0
1
0
2
3
2
0


Какие из классов являются dataclass с какими параметрами? Почему?

<b><font color="#FF69B4"> 


Address - frozen dataclass
(значение, которое не должно меняться после создания. Если адрес изменился - это уже другой дом)

Person - dataclass  
(ФИО человека может меняться, если он подаст соответствующие заявления)

Violation - dataclass
(статус нарушения может меняться)

Household - dataclass
(Домовладение - это изменяемая сущность: добавляются/удаляются жильцы, появляются новые нарушения, оплачиваются штрафы.)


slots использован везде, т к нет динамики и он оптимизирует память
 </font></b>

## Отчаянные домохозяйки - 2

Реализуй класс HOA:
- унаследуй его от `Sequence` из `collections.abc`
- этот класс будет хранить информацию о всех домах (`household`), но итерироваться только по тем, в которых есть жильцы.
- реализуй все методы, необходимые для того, чтобы код заработал.

In [51]:
from dataclasses import dataclass, field
from typing import List, Optional, Iterator
from collections.abc import Sequence
from datetime import datetime

@dataclass(slots=True)
class HOA(Sequence):
    name: str
    households: List[Household] = field(default_factory=list)
    
    def __getitem__(self, index: int) -> Household:
        households_with_residents = self._get_households_with_residents()
        return households_with_residents[index]
    
    def __len__(self) -> int:
        return len(self._get_households_with_residents())
    
    def __contains__(self, item: object) -> bool:
        if not isinstance(item, Household):
            return False
        return item in self._get_households_with_residents()
    
    def __iter__(self) -> Iterator[Household]:
        return iter(self._get_households_with_residents())
    
    def __reversed__(self) -> Iterator[Household]:
        return reversed(self._get_households_with_residents())
    
    def index(self, value: Household, start: int = 0, stop: Optional[int] = None) -> int:
        households_with_residents = self._get_households_with_residents()
        if stop is None:
            return households_with_residents.index(value, start)
        return households_with_residents.index(value, start, stop)
    
    def _get_households_with_residents(self) -> List[Household]:
        return [household for household in self.households if household.has_residents]
    
    @property
    def total_households(self) -> int:
        return len(self.households)
    
    @property
    def total_residents(self) -> int:
        return sum(household.residents_count for household in self.households)

    def add_household(self, household: Household) -> None:
        self.households.append(household)
    
    def find_household_by_address(self, address: Address) -> Optional[Household]:
        for household in self.households:
            if household.address == address:
                return household
        return None

    @property
    def total_unpaid_fines(self) -> float:
        return sum(household.total_fines for household in self.households)
    
    def get_empty_households(self) -> List[Household]:
        return [household for household in self.households if not household.has_residents]



In [54]:
household1.has_residents

True

In [56]:
address3 = Address("Садовая", "7")

household3 = Household(
    address=address3,
    owner=person1,
    residents=[],
)

hoa = HOA("Садовое товарищество 'Рубашка Николая'")
hoa.add_household(household1)
hoa.add_household(household2)
hoa.add_household(household3)
    

print(hoa.name)
print(f"Всего домовладений: {hoa.total_households}")
    
print("\nИтерация по домовладениям с жильцами:")
for i, household in enumerate(hoa):
    print(f"{i+1}. {household.address} - {household.residents_count} жильцов")

print(f"\nhousehold1 в НОА: {household1 in hoa}")
print(f"household3 в НОА: {household3 in hoa}")
    
print(f"\nОбратная итерация:")
for household in reversed(hoa):
    print(household.address)
    

print(f"\nВсего жильцов: {hoa.total_residents}")
print(f"Неоплаченные штрафы: {hoa.total_unpaid_fines} руб.")
    
print("\nПустые домовладения:")
for empty_house in hoa.get_empty_households():
    print(f"- {empty_house.address}")

Садовое товарищество 'Рубашка Николая'
Всего домовладений: 3

Итерация по домовладениям с жильцами:
1. ул. Ленина, д. 15, корп. А - 3 жильцов

household1 в НОА: True
household3 в НОА: False

Обратная итерация:
ул. Ленина, д. 15, корп. А

Всего жильцов: 3
Неоплаченные штрафы: 2000.0 руб.

Пустые домовладения:
- ул. Пушкина, д. 42
- ул. Садовая, д. 7


# Борьба с кэшированием

## cache - 1

Допиши строчки так, чтобы декоратор `@cache`, сохраняющий предыдущий вызов функции и возвращающий его, если пришёл тот же аргумент

In [61]:
def cache(f):
    last_arg = None
    last_res = None
    def wrapper(x):
        nonlocal last_arg
        nonlocal last_res
        if x == last_arg:
            return last_res
        else:
            last_arg = x
            last_res = f(x)
            return last_res
    return wrapper

@cache
def f(x):
    for i in range(100000):
        a = i ** 2
    return x ** 2

print(f(1))
print(f(1))

1
1


## cache - 2

Переделай декоратор `@cache` из п.1 так, чтобы он мог работать для функций с произвольным числом аргументов

In [62]:
def cache(f):
    last_args = None
    last_kwargs = None
    last_res = None
    
    def wrapper(*args, **kwargs):
        nonlocal last_args
        nonlocal last_kwargs
        nonlocal last_res
        if (args == last_args) and (kwargs == last_kwargs):
            return last_res
        else:
            last_args = args
            last_kwargs = kwargs
            last_res = f(*args, **kwargs)
            return last_res
    
    return wrapper

@cache
def f(x, y):
    for i in range(100000):
        a = i ** 2
    return (x ** 2) * y

print(f(1, 2))
print(f(1, 2))

2
2


## LRU_cache - 1

Реализуй декоратор `@lru_cache`: он будет хранить результаты последних 5 уникальных вызовов функции.

- Если мы вызвали то, что уже лежит в наборе последних 5 уникальных вызовов, то поднимаем "вверх" этот вызов до последнего и выводим результат
- Если мы вызвали то, что не лежит в наборе:

- если длина набора меньше 5, то просто добавляем последний вызов, считаем значение и выводим
- если длина набора 5, то удаляем самый старый вызов, добавляем наш вызов с учётом посчитанного значения и выводим



In [65]:
def lru_cache(f):
    cache = {}
    order = []
        
    def wrapper(*args, **kwargs):
        key = (args, frozenset(kwargs.items()))
            
        if key in cache:
            order.remove(key)
            order.append(key)
            return cache[key]
        else:
            result = f(*args, **kwargs)
                
            if len(cache) < 5:
                cache[key] = result
                order.append(key)
            else:
                oldest_key = order.pop(0)
                del cache[oldest_key]
                cache[key] = result
                order.append(key)
                
            return result
    return wrapper


## LRU_cache - 2

Возьми код из предыдущей ячейки и реализуй `@lru_cache(n)`, которых хранит результаты последних n уникальных вызовов функции.

In [64]:
def lru_cache(maxsize):
    def decorator(f):
        cache = {}
        order = []
        
        def wrapper(*args, **kwargs):
            key = (args, frozenset(kwargs.items()))
            
            if key in cache:
                order.remove(key)
                order.append(key)
                return cache[key]
            else:
                result = f(*args, **kwargs)
                
                if len(cache) < maxsize:
                    cache[key] = result
                    order.append(key)
                else:
                    oldest_key = order.pop(0)
                    del cache[oldest_key]
                    cache[key] = result
                    order.append(key)
                
                return result
        return wrapper
    return decorator

Можем ли мы корректно удалять объекты, которые возвращает lru_cache? Почему?


<b><font color="#FF69B4"> Мы не можем корректно удалять объекты из lru_cache, потому что не знаем, сколько ссылок на объект существует и не знаем, когда все ссылки перестанут использоваться (т е если есть два объекта, для одного из которых функция значение считала, а для второго вывела закешированное от первого, то при изменении одного объекта изменится и второй)</font></b>

## LRU_cache - 3

Реализуй `LRU_cache(n)` через `weakref`, чтобы можно было удалять элементы.

In [71]:
import weakref
from collections import OrderedDict

class WeakRefableList:  # мне питоник написал что не поддерживает weakref для списков, пришлось выкручиваться
    def __init__(self, data):
        self.data = data
    
    def __repr__(self):
        return repr(self.data)
    
    def __eq__(self, other):
        if isinstance(other, WeakRefableList):
            return self.data == other.data
        return self.data == other

def lru_cache(maxsize=5):
    def decorator(f):
        cache = OrderedDict()
        
        def wrapper(*args, **kwargs):
            key = (args, frozenset(kwargs.items()))
            
            if key in cache:
                result_ref = cache.pop(key)
                result_wrapper = result_ref()
                if result_wrapper is not None:
                    cache[key] = result_ref
                    return result_wrapper.data
                else:
                    result = f(*args, **kwargs)
                    result_wrapper = WeakRefableList(result)
                    cache[key] = weakref.ref(result_wrapper)
                    return result
            else:
                result = f(*args, **kwargs)
                result_wrapper = WeakRefableList(result)
                result_ref = weakref.ref(result_wrapper)
                
                if len(cache) >= maxsize:
                    # ё=ъёпаима - оставлю для истории. Это кот по клавиатуре прошелся
                    cache.popitem(last=False)
                
                cache[key] = result_ref
                return result
        
        return wrapper
    return decorator

# Скитания Palomar-1 часть 2

## Генератор и/или декоратор Palomar - 1

Возьми датасет про Palomar-1. Давай сымитируем передачу данных между сервисами - напиши класс, который в `__init__` получает название файла (путь до файла), в `read_from_file` записывает все данные из файла в некоторый атрибут, а в `get_lines(n)` возвращает следующие n линий из файла в формате np.array (такой вот итератор без `__iter__` и `__next__`).

Если линии кончились - пусть метод выбрасывает ошибку (StopIteration).

Протестируй работу класса.

In [73]:
import numpy as np

class DataReader:
    def __init__(self, filename):
        self.filename = filename
        self.data = None
        self.current_index = 0
    
    def read_from_file(self):
        with open(self.filename, 'r') as file:
            lines = file.readlines()
            has_header = False
            if lines:
                first_line = lines[0].strip().split()
                try:
                    float(first_line[0])
                except ValueError:
                    has_header = True
            data_lines = lines[1:] if has_header else lines
            self.data = []
            for line in data_lines:
                line = line.strip()
                if line:
                    values = [float(x) for x in line.split()]
                    self.data.append(values)
            self.data = np.array(self.data)
            self.current_index = 0
    
    def get_lines(self, n):
        if self.data is None:
            raise ValueError("Данных нету(")
        if self.current_index >= len(self.data):
            raise StopIteration("Данных мало(")
        end_index = min(self.current_index + n, len(self.data))
        result = self.data[self.current_index:end_index]
        self.current_index = end_index
        return result


In [85]:
reader = DataReader("Pal1_new.dat")
reader.read_from_file()
batch_num = 1
while True:
    try:
        batch = reader.get_lines(800)
        print(f"Партия {batch_num}: {len(batch)} строк")
        print(F"Первая строка блока: {np.array2string(batch[0], separator=';', precision=4)}")
        batch_num += 1
    except StopIteration:
        print("Достигнут конец файла")
        break


Партия 1: 800 строк
Первая строка блока: [  0.    ;-13.4307;  6.4598;  2.9211; 62.2444;208.7595;-23.2642]
Партия 2: 800 строк
Первая строка блока: [ 800.    ; -11.6909;  -8.2751;  -1.1206;-111.2145; 195.5002;  70.6211]
Партия 3: 800 строк
Первая строка блока: [1600.    ;  -4.8749; -15.6137;  -2.9148;-187.4511;  57.2455; -34.5734]
Партия 4: 800 строк
Первая строка блока: [2400.    ;   5.896 ; -12.233 ;   2.7311;-204.2296;-120.0029; -26.8959]
Партия 5: 800 строк
Первая строка блока: [ 3.2000e+03; 1.5705e+01;-2.1382e+00;-9.9785e-01;-4.8284e+01;-1.9755e+02;
  6.7108e+01]
Партия 6: 800 строк
Первая строка блока: [ 4.0000e+03; 1.4034e+01; 6.2202e+00;-3.0582e+00; 1.0549e+02;-1.8168e+02;
 -2.9985e+01]
Партия 7: 800 строк
Первая строка блока: [ 4.8000e+03; 1.9280e+00; 1.3584e+01; 2.8878e+00; 2.2907e+02;-4.8874e+01;
 -2.5064e+01]
Партия 8: 800 строк
Первая строка блока: [ 5.6000e+03;-9.0623e+00; 1.3947e+01;-1.8713e-01; 1.6218e+02; 1.0416e+02;
  7.0077e+01]
Партия 9: 800 строк
Первая строка блока

## Генератор и/или декоратор Palomar - 2

Напиши функцию palomar_mean, которая бы принимала `np.array` из нескольких строк из Palomar'a и считает среднее по каждому из них, возвращая другой `np.array`.

In [86]:
import numpy as np

def palomar_mean(data):
    if data.ndim == 1:
        return np.array([np.mean(data)])
    return np.mean(data, axis=1)

In [87]:
reader = DataReader("Pal1_new.dat")
reader.read_from_file()
test_data = reader.get_lines(10)
means = palomar_mean(test_data)
print(means)
single_row = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
print(palomar_mean(single_row))

[34.81284298 35.02627372 35.23213678 35.43046549 35.62129364 35.80465546
 35.98058559 36.14911901 36.31029098 36.4641371 ]
[3.]


## Генератор и/или декоратор Palomar - 3

Напиши декоратор `generate_rows(n)`, который:

- принимает на вход параметр n (по сколько строк данных он должен брать) - для метода `get_lines` у класса `DataReader`

- принимает функцию f, а на выходе выдаёт генератор, который возвращает обработанные функцией f строки `np.array`.

In [88]:
import numpy as np
from functools import wraps

def generate_rows(n):
    def decorator(func):
        @wraps(func)
        def wrapper(reader, *args, **kwargs):
            original_position = reader.current_index
            reader.current_index = 0
            while True:
                try:
                    lines = reader.get_lines(n)
                    result = func(lines, *args, **kwargs)
                    yield result
                except StopIteration:
                    break
            reader.current_index = original_position
        return wrapper
    return decorator

In [94]:
reader = DataReader("Pal1_new.dat")
reader.read_from_file()

@generate_rows(5000)
def filter_high_values(lines, threshold=50):
    high_value_mask = np.any(lines > threshold, axis=1)
    return lines[high_value_mask]
    
generator = filter_high_values(reader, threshold=50)
total_filtered = 0
for filtered_batch in generator:
    if len(filtered_batch) > 0:
        print(f"Найдено строк: {len(filtered_batch)}")
        total_filtered += len(filtered_batch)
    
print(f"Всего отфильтровано строк: {total_filtered}")
    

@generate_rows(5000)
def aggregate_data(lines):
    return {
        'batch_mean': np.mean(lines),
        'batch_size': lines.shape[0],
    }
    
print("статистика по партиям:")
generator = aggregate_data(reader)
    
for i, agg_stats in enumerate(generator):
    print(f"Партия {i+1}: среднее={agg_stats['batch_mean']:.2f}, "
          f"строк={agg_stats['batch_size']}")

Найдено строк: 5000
Найдено строк: 5000
Найдено строк: 5000
Найдено строк: 5000
Найдено строк: 5000
Найдено строк: 5000
Найдено строк: 5000
Найдено строк: 5000
Найдено строк: 5000
Найдено строк: 5000
Найдено строк: 1
Всего отфильтровано строк: 50001
статистика по партиям:
Партия 1: среднее=356.71, строк=5000
Партия 2: среднее=1072.32, строк=5000
Партия 3: среднее=1785.41, строк=5000
Партия 4: среднее=2499.04, строк=5000
Партия 5: среднее=3215.06, строк=5000
Партия 6: среднее=3928.77, строк=5000
Партия 7: среднее=4641.90, строк=5000
Партия 8: среднее=5357.24, строк=5000
Партия 9: среднее=6072.08, строк=5000
Партия 10: среднее=6785.25, строк=5000
Партия 11: среднее=7185.07, строк=1


# Большой брат `observer` за тобой

## Observer - 1


Напиши класс `ObservableMixin`. Он должен:

- Переопределять `__init__`, при этом не нарушать наследование (не забудь про super())
- Создавать словарь с observer'ами в init'е - по сути ссылками на функции, которые надо будет дёргать. Ключ - строка - название события, по которому надо начать оповещать. Значение - список из функций, которые надо будет вызвать в случае события.

У него должны быть методы:
- add_observer - добавление обзервера (функций для оповещения) для выбранных событий

- delete_observer - удаление обзервера (функции для оповещения) среди всех событий

- clear - очищение всех обзерверов

- notify_observers - запускает все функции из списка, который хранится по данному ключу

In [95]:
class ObservableMixin:
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._observers: Dict[str, List[Callable]] = {}
    
    def add_observer(self, event, observer):
        if event not in self._observers:
            self._observers[event] = []
        if observer not in self._observers[event]:
            self._observers[event].append(observer)
    
    def delete_observer(self, observer):
        for event in self._observers:
            if observer in self._observers[event]:
                self._observers[event].remove(observer)
    
    def clear(self, event=None):
        if event is None:
            self._observers.clear()
        elif event in self._observers:
            self._observers[event].clear()
    
    def notify_observers(self, event, *args, **kwargs):
        if event in self._observers:
            observers = self._observers[event].copy()
            for observer in observers:
                observer(*args, **kwargs)

In [98]:
class Button(ObservableMixin):
    def __init__(self, label: str):
        super().__init__()
        self.label = label
        self.clicked = False

    def click(self) -> None:
        self.clicked = True
        self.notify_observers("click", self, self)

In [99]:
def on_button_click(sender, button):
    print(f"Button '{button.label}' was clicked!")

button = Button("Submit")
button.add_observer("click", on_button_click)

button.click()

# это код для проверки - но можешь придумать свой код!

Button 'Submit' was clicked!


## Observer - 2

Теперь реализуй декоратор observable, который на вход принимает класс, который нужно сделать observable - реализует Observable (по своей сути ObservableMixin из задания выше), наследуемый от него, и переопределяет методы так, чтобы они после своего выполнения запускали `notify_observers`.

In [104]:
def observable(cls):
    class ObservableClass(ObservableMixin, cls):
        def __init__(self, *args, **kwargs):
            ObservableMixin.__init__(self)
            cls.__init__(self, *args, **kwargs)
        
        def __getattribute__(self, name):
            attr = object.__getattribute__(self, name)
            if callable(attr) and not name.startswith('_') and name not in [
                'add_observer', 'delete_observer', 'clear', 'notify_observers',
            ]:
                def wrapper(*args, **kwargs):
                    self.notify_observers(
                        f"before_{name}",
                        method_name=name,
                        args=args,
                        kwargs=kwargs,
                        instance=self
                    )
                    result = attr(*args, **kwargs)
                    self.notify_observers(
                        f"after_{name}",
                        method_name=name,
                        args=args,
                        kwargs=kwargs,
                        result=result,
                        instance=self
                    )  
                    return result
                return wrapper
            return attr
    ObservableClass.__name__ = f"Observable{cls.__name__}"
    ObservableClass.__qualname__ = f"Observable{cls.__qualname__}"
    return ObservableClass

In [105]:
@observable
class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        return self.balance

def logger(method_name, *args, **kwargs):
    print(f"Method '{method_name}' called with args: {args}, kwargs: {kwargs}")

my_acc = BankAccount(1000)
my_acc.add_observer("deposit", logger)
my_acc.deposit(2000)

# тут какой-то код для проверки, но ты можешь придумать свой код для проверки :)

3000

Теперь:

1. Сравни эти две реализации. Какая лучше?
2. Предложи свои идеи по улучшению/доработке. Что можно было бы реализовать по-другому?

<b><font color="#FF69B4"> 
1. Класс ObservableMixin и наследование от него хорошо тем, что методы ObservableMixin вызываются явно, соответствеенно менгьше путаницы при чтении кода. Но при этом плохо то, что в наследуемый от него класс надо добавлять каждый раз методы ObservableMixin заново. Можно легко забыть это сделать, ну и ещё это дублирование кода.

Декоратор хорош тем, что не надо в классе самого объекта прописываь что-либо от Observable, соответственно нет повторений кода и нечего забывать. Если нужно создавать много классов которые в итоге будут Observable декоратор подходит лучше. Но для декоратора не так понятно в какой момент вообще отправляются уведомления Observable и для сложных методов классов из-за этого может быть сильно затруднена отладка.

2. Можно добавить асинхронность для наблюдателей. В синхронной версии, когда вызывается уведомление наблюдателей, основной поток выполнения блокируется до тех пор, пока все наблюдатели не завершат свою работу. Если какой-то наблюдатель выполняет длительную операцию (например, запрос к базе данных или внешнему API), всё приложение "зависает" на это время. В асинхронной версии основной поток может продолжать работу, пока наблюдатели выполняют свои задачи в фоне.

Можно добавить отчеты о состоянии и логирование. Для более прозрачной работы. Такая система позволяет видеть, сколько уведомлений отправляется, как часто возникают события, нет ли аномалий в работе. Это дает оперативную информацию о "здоровье" системы наблюдения и помогает быстро обнаруживать проблемы.
 </font></b>

# Удивительный `descriptor` страны Оз

Дескрипторы в Python - классы, в которых реализованы методы `__get__`, `__set__` или `__del__`. Эти методы вызываются, когда мы обращаемся к объекту данного класса как к атрибуту у другого класса.

In [ ]:
# пример, тут делать ничего не надо

class Ten:
    def __get__(self, *args, **kwargs):
        return 10

class Numbers:
    a = 5
    ten = Ten()


print(Numbers.a, Numbers.ten)

5 10


Оказывается, что многие фишки реализованы через дескрипторы в питоне. Ваша задача - написать свои аналоги (быть может, упрощенные версии того, как этот код работает на самом деле)

## @my_property

Реализуй @my_property - аналог @property - через дескрипторы

In [106]:
class MyProperty:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc:
            self.__doc__ = doc
        elif fget:
            self.__doc__ = fget.__doc__
        else:
            None
        if fget:
            self.name = fget.__name__
        else:
            None
    
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError(f"атрибут '{self.name}' нечитаем")
        return self.fget(obj)
    
    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError(f"атрибут '{self.name}' не устанавливается")
        self.fset(obj, value)
    
    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError(f"атрибут '{self.name}' не удаляется")
        self.fdel(obj)
    
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)
    
    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)
    
    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)


def my_property(fget=None, fset=None, fdel=None, doc=None):
    if fget is None:
        def decorator(func):
            return MyProperty(func, fset, fdel, doc)
        return decorator
    else:
        return MyProperty(fget, fset, fdel, doc)

In [109]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age
        self._email = None

    @my_property
    def name(self):
        return self._name

    @my_property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if value < 0:
            raise ValueError("Возраст не может быть отрицательным")
        self._age = value

    @my_property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, value):
        self._email = value
    
    @email.deleter
    def email(self):
        self._email = None

    @my_property
    def is_adult(self):
        return self._age >= 18


person = Person("Иван", 25)
print(person.name)
print(person.age)
print(person.is_adult)
person.age = 30
print(person.age)
person.email = "ivan@phystech.edu"
print(person.email)
del person.email
print(person.email)

Иван
25
True
30
ivan@phystech.edu
None


## @my_classmethod

Реализуй @my_classmethod - аналог @classmethod - через дескрипторы

In [110]:
class MyClassMethod:
    def __init__(self, func):
        self.func = func
        self.__doc__ = func.__doc__
        self.__name__ = func.__name__
    
    def __get__(self, obj, objtype=None):
        if objtype is None:
            objtype = type(obj)
        def cls_wrapper(*args, **kwargs):
            return self.func(objtype, *args, **kwargs)
        cls_wrapper.__doc__ = self.__doc__
        cls_wrapper.__name__ = self.__name__
        cls_wrapper.__qualname__ = getattr(self.func, '__qualname__', self.__name__)
        
        return cls_wrapper


def my_classmethod(func):
    return MyClassMethod(func)

In [111]:
class Person:
    species = "Homo Sapiens"
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    @my_classmethod
    def get_species(cls):
        return cls.species
    
    @my_classmethod
    def create_adult(cls, name):
        return cls(name, age=18)
    
    @my_classmethod
    def from_string(cls, string):
        name, age = string.split(',')
        return cls(name, int(age))

    def get_info(self):
        return f"{self.name}, {self.age} лет"

Person.get_species()


'Homo Sapiens'

## @my_staticmethod

Реализуй @my_staticmethod - аналог @staticmethod - через дескрипторы

In [112]:
class MyStaticMethod:
    def __init__(self, func):
        self.func = func
        self.__doc__ = func.__doc__
        self.__name__ = func.__name__
        self.__qualname__ = getattr(func, '__qualname__', func.__name__)
    
    def __get__(self, obj, objtype=None):
        return self.func


def my_staticmethod(func):
    return MyStaticMethod(func)

In [114]:
class MathOperations:
    PI = 3.14159
    
    def __init__(self, value=0):
        self.value = value
    
    @my_staticmethod
    def add(a, b):
        return a + b
    
    @my_staticmethod
    def multiply(a, b):
        return a * b
    
    @my_staticmethod
    def circle_area(radius):
        return MathOperations.PI * radius * radius

m = MathOperations()
print(m.add(1, 2))
print(m.multiply(3, 4))
print(m.circle_area(5))

3
12
78.53975


# Джонни! `Singleton`'ы на деревьях!

## Singleton - Dec1

Допиши код внизу, чтобы он стал рабочим

In [117]:
def singleton(cls):
    _instances = {} 

    def return_obj(*args, **kwargs):
        if cls not in _instances:
            _instances[cls] = cls(*args, **kwargs)
        return _instances[cls]
    return return_obj

@singleton
class House:
    def __init__(self, name):
        self.name = name

    def whose(self):
        print(f"It's {self.name}'s house!")

my_name = 'Tatiana'
h1 = House(my_name)
h1.whose()
h2 = House(f"Not {my_name}")
h2.whose()
print(h1 is h2)

It's Tatiana's house!
It's Tatiana's house!
True


Данная реализация позволяет нам вызывать повторно `__init__`?

<b><font color="#FF69B4"> Нет. Если один объект класса есть то второй не создается. </font></b>

## Singleton - Dec2

Реализуй декоратор так, чтобы он позволял не создавать новый объект, но вызывал повторно `__init__`

In [119]:
def singleton_reinit(cls):
    _instances = {}

    def return_obj(*args, **kwargs):
        if cls not in _instances:
            _instances[cls] = cls.__new__(cls)
        _instances[cls].__init__(*args, **kwargs)
        return _instances[cls]
    
    return return_obj

@singleton_reinit
class House:
    def __init__(self, name):
        self.name = name
        print(f"init вызван для {name}")

    def whose(self):
        print(f"It's {self.name}'s house!")

my_name = 'Tatiana'
h1 = House(f"Not {my_name}")
h1.whose()
h2 = House(f"{my_name}")
h2.whose()
print(h1 is h2)

init вызван для Not Tatiana
It's Not Tatiana's house!
init вызван для Tatiana
It's Tatiana's house!
True


## Singleton - Dec3

Обязательно ли нам нужен мутабельный объект (`dict`, `list`) для реализации декоратора `singleton`?

Реализуй декоратор через просто хранение переменной `_instance`.

In [122]:
def singleton(cls):
    _instance = None
    def return_obj(*args, **kwargs):
        nonlocal _instance
        if not _instance:
            _instance = cls(*args, **kwargs)
        return _instance
    return return_obj

@singleton
class House:
    def __init__(self, name):
        self.name = name

    def whose(self):
        print(f"It's {self.name}'s house!")

my_name = 'Tatiana'
h1 = House(my_name)
h1.whose()
h2 = House(f"Not {my_name}")
h2.whose()
print(h1 is h2)

It's Tatiana's house!
It's Tatiana's house!
True


Корректен ли такой код? Если да, то почему? Если нет - приведи контрпример, где некорректен


<b><font color="#FF69B4"> Он корректен пока мы используем singleton для одного класса. Если нужен один singleton на несколько классов, то нужен мутабельный объект, иначе мы никак не сможем хранить экземляры разных класов в одной не мутабельной переменной </font></b>

## Singleton - Dec4

Получается ли удалять объекты корректно с таким декоратором? Почему?

Реализуй декоратор через `weakref`, чтобы поддерживать удаление

In [127]:
def singleton_with_reinit_del(cls):
    _instances = {}

    def return_obj(*args, **kwargs):
        if cls not in _instances:
            _instances[cls] = cls.__new__(cls)
        
        _instances[cls].__init__(*args, **kwargs)
        return _instances[cls]
    
    def delete_instance():
        if cls in _instances:
            instance = _instances[cls]
            if hasattr(instance, 'cleanup'):
                instance.cleanup()
            del _instances[cls]
            return True
        return False
    
    def reset_instance(*args, **kwargs):
        delete_instance()
        return return_obj(*args, **kwargs)
    
    def get_instance():
        return _instances.get(cls)
    
    def is_initialized():
        return cls in _instances

    return_obj.delete = delete_instance
    return_obj.reset = reset_instance
    return_obj.get_instance = get_instance
    return_obj.is_initialized = is_initialized
    
    return return_obj


@singleton_with_reinit_del
class House:
    def __init__(self, name):
        self.name = name
        print(f"init для '{name}'")
    
    def cleanup(self):
        print(f"del для {self.name}")
    
    def whose(self):
        print(f"It's {self.name}'s house!")


h1 = House("Tatiana")
h1.whose()

h2 = House("Migal")
h2.whose()
print(h1 is h2)
House.delete()
print(House.is_initialized())

init для 'Tatiana'
It's Tatiana's house!
init для 'Migal'
It's Migal's house!
True
del для Migal
False


## Singleton - Meta1

Допиши код внизу, чтобы он был рабочим

In [128]:
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class House(metaclass=SingletonMeta):
    def __init__(self):
        print("It's my House!")

h1 = House()
h2 = House()

It's my House!


## Singleton - Meta2

Можно ли реализовать код выше не через mutable объект (dict), а через изменение одной переменной?

1. Попробуй сделать это через `nonlocal _instance`. Получается?

In [131]:
class SingletonMeta(type):
    _instances = None

    def __call__(cls, *args, **kwargs):
        if not cls._instances:
            cls._instances = super().__call__(*args, **kwargs)
        return cls._instances

class House(metaclass=SingletonMeta):
    def __init__(self):
        print("It's my House!")

h1 = House()
h2 = House()

It's my House!


Объясните, почему получилось или не получилось.

 через nonlocal не получится конечно, потому что для атрибута класса он не работает. Зато будет работать просто через cls.nonlocal

2. Попробуй сделать это через `SingletonMeta._instance`. Корректен ли код?

Если нет - приведи ситуацию, когда код ведёт себя некорректно

In [132]:
class SingletonMeta(type):
    _instances = None

    def __call__(cls, *args, **kwargs):
        if not SingletonMeta._instances:
            cls._instances = super().__call__(*args, **kwargs)
        return cls._instances

class House(metaclass=SingletonMeta):
    def __init__(self):
        print("It's my House!")

h1 = House()
h2 = House()

It's my House!
It's my House!


Опиши, что пошло не так. Можно ли было так сделать, когда мы писали через декоратор? Почему?

<b><font color="#FF69B4"> Использование SingletonMeta._instance корректно только если иметь всего один класс с этим метаклассом. Для поддержки нескольких классов нужен мутабельный _instances </font></b>

## Singleton - Meta3

Какие есть минусы у текущей реализации `Singleton` через Метаклассы? Позволяет ли он удалять объекты? Почему?



<b><font color="#FF69B4"> Основной минус - поддержка всего одного класса. Объекты тоже удалять нельзя, так как для этого нет методов </font></b>

Реализуй удаление через метод `delete_object` (т.е. мы глобально хотим вызывать `h.delete_object()`). Где он (метод) должен находиться, чтобы он работал?

In [133]:
class SimpleSingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
            instance.delete_object = lambda: cls._instances.pop(cls, None)
            
        return cls._instances[cls]

class House(metaclass=SimpleSingletonMeta):
    def __init__(self):
        print("It's my House!")


h = House()
h.delete_object()

It's my House!


Объясни, как работает твоё решение

<b><font color="#FF69B4">  delete_object создается в метаклассе и добавляется к каждому объекту. После чего его можно вызывать как объект.delete_object(). Метод delete_object использует замыкание - он помнит класс через замыкание и может удалить объект из _instances  </font></b>

## Singleton - Meta4

Реализуй хранение в метаклассе через `weakref`.

In [140]:
import weakref

class SingletonMetaWeakRef(type):
    _instances = weakref.WeakValueDictionary()

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
            instance.delete_object = lambda: cls._delete_instance()
            
        return cls._instances[cls]
    
    def _delete_instance(cls):
        if cls in cls._instances:
            del cls._instances[cls]
            return True
        return False

class House(metaclass=SingletonMetaWeakRef):
    def __init__(self, name="Unknown"):
        self.name = name
        print(f"House for {name} created")


h1 = House("Tatiana")
print(h1.name)

del h1

h2 = House("Migal")
print(h2.name)

House for Tatiana created
Tatiana
House for Migal created
Migal


Какое решение лучше для удаления объектов - в Meta3 (через некий метод) или в Meta4 (через `weakref`)?

Сравните их. В какой ситуации какой может быть лучше?


<b><font color="#FF69B4"> WeakRef лучше в большинстве случаев, т к обеспечивает автоматическое управление памятью и предотвращает утечки. Объекты удаляются, когда теряются все сильные ссылки, что безопасно

Явное удаление через метод лучше когда нужен полный контроль над временем жизни объектов
 </font></b>

## Singleton'ы. Сравнение

Сравни реализации Singleton'ов через декораторы и через метаклассы. Какие есть плюсы и минусы? Что лучше?

<b><font color="#FF69B4"> Декораторы проще в понимании и использовании, они подходят для быстрого прототипирования. Легко добавлять дополнительную логику. Но могут быть проблемы с наследованием.

Метаклассы обеспечивают более чистую реализацию на уровне создания классов, лучше работают с наследованием. Но сложнее для чтения и отладки.

 </font></b>

# `__slots__` этого `__attribute_name__` казино

Как мы знаем, `__slots__` лучше, чем `__dict__` в контексте времени и памяти, но убивает динамичность атрибутов.

Объясни своими словами, какая проблема возникнет, если мы захотим сделать метакласс, который будет нам перестраивать классы таким образом?

<b><font color="#FF69B4"> slots нельзя настраивать после построения тела класса(не в опреденениии класса). Если изменить его после того, как вызовется метакласс, то все порушится, т к питон считает что решение о структуре slots уже принято. </font></b>

## MakeSlottable - Meta1

Давай попробуем передавать аргументы в метаклассы. Наша первая идея - поступить как с декораторами - сделать функцию, которая станет замыканием для наших методов класса; таким образом, на каждый вызов у нас будет свой метакласс.

In [142]:
def MakeSlottable(*slot_names):
    class SlottableMeta(type):
        def __new__(cls, name, bases, namespace):
            namespace['__slots__'] = slot_names
            return super().__new__(cls, name, bases, namespace)
    
    return SlottableMeta

In [144]:
class A(metaclass = MakeSlottable("a", "b", "c")):
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

In [145]:
obj = A(1, 2, 3)

print(obj.a)
print(obj.b)
print(obj.c)

print(A.__slots__)

try:
    obj.d = 4
except AttributeError as e:
    print(f"все как должно быть: {e}")

print(f"Есть dict: {hasattr(obj, '__dict__')}")


1
2
3
('a', 'b', 'c')
все как должно быть: 'A' object has no attribute 'd' and no __dict__ for setting new attributes
Есть dict: False


## MakeSlottable - Meta2

Теперь давай переделаем этот метакласс: будем передавать параметры через kwargs при наследовании.

In [153]:
class MakeSlottable(type):
    def __new__(cls, name, bases, namespace, **kwargs):
        slots = kwargs.pop('slots_names', None)
        namespace['__slots__'] = slots
        return super().__new__(cls, name, bases, namespace)
    
    def __init__(self, name, bases, namespace, **kwargs):
        kwargs.pop('slots', None)
        super().__init__(name, bases, namespace)



In [154]:
class A(metaclass = MakeSlottable, slots_names = ["a", "b", "c"]):
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

In [156]:
obj_a = A(10, 20, 30)
print(f"A.__slots__ = {A.__slots__}")
print(f"obj_a.a = {obj_a.a}, obj_a.b = {obj_a.b}")

A.__slots__ = ['a', 'b', 'c']
obj_a.a = 10, obj_a.b = 20


Сравни эти два способа реализации передачи параметров в метакласс. Какой лучше? Найди **минимум 2** различных минуса у первого (Meta1) способа.

<b><font color="#FF69B4"> Способ Meta1 хуже, т к нарушает семантику наследования - выглядит как вызов функции, а не объявление класса и имеет сложность с наследованием - нельзя комбинировать с другими метаклассами или базовыми классами без костылей

Kwargs-способ чище и гибче, соответствует стандартному Python. </font></b>

## MakeSlottable - Meta3



Давай попробуем реализовать без каких-либо параметров - метакласс, который бы из этой записи (через атрибуты класса):
```
class MyClass:
    x: int
    y: int
    z: int
```

`__slots__` с `x`, `y`, `z` вместо `__dict__`

In [160]:
class SlotsFromAnnotations(type):
    def __new__(cls, name, bases, namespace):
        annotations = namespace.get('__annotations__', {})
        for base in bases:
            if hasattr(base, '__annotations__'):
                annotations.update(base.__annotations__)
        namespace['__slots__'] = tuple(annotations.keys())
        return super().__new__(cls, name, bases, namespace)

In [163]:
class A(metaclass = SlotsFromAnnotations):
    a: int
    b: int  
    c: int

    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

In [164]:
obj = A(1, 2, 3)
print(f"obj.a = {obj.a}, obj.b = {obj.b}, obj.c = {obj.c}")
print(f"A.__slots__ = {A.__slots__}")
try:
    obj.d = 4
except AttributeError as e:
    print(f"все как надо: {e}")
print(f"Есть __dict__: {hasattr(obj, '__dict__')}")


obj.a = 1, obj.b = 2, obj.c = 3
A.__slots__ = ('a', 'b', 'c')
все как надо: 'A' object has no attribute 'd' and no __dict__ for setting new attributes
Есть __dict__: False


## MakeSlottable - Meta4

Давай попробуем теперь использовать `inspect.sig` из модуля `inspect` для получения атрибутов init'а, чтобы на их основе сделать `slots`.

In [165]:
import inspect

class SlotsFromInit(type):
    def __new__(cls, name, bases, namespace):
        init_method = namespace.get('__init__')
        if init_method and callable(init_method):
            sig = inspect.signature(init_method)
            param_names = [param.name for param in sig.parameters.values() 
                          if param.name != 'self']
            if param_names:
                namespace['__slots__'] = tuple(param_names)
        return super().__new__(cls, name, bases, namespace)


In [166]:
class A(metaclass = SlotsFromInit ):
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

In [167]:
obj = A(1, 2, 3)
print(f"obj.a = {obj.a}, obj.b = {obj.b}, obj.c = {obj.c}")
print(f"A.__slots__ = {A.__slots__}")

try:
    obj.d = 4
except AttributeError as e:
    print(f"так надо: {e}")

print(f"Есть __dict__: {hasattr(obj, '__dict__')}")


obj.a = 1, obj.b = 2, obj.c = 3
A.__slots__ = ('a', 'b', 'c')
так надо: 'A' object has no attribute 'd' and no __dict__ for setting new attributes
Есть __dict__: False


Сравни два способа реализации MakeSlottable без аргументов (Meta3 и Meta4). Какой лучше? Для каких целей?

Сравни их со способами с аргументами. Какой бы ты выбрал, если бы нужно было сделать один вариант для прода?

<b><font color="#FF69B4"> 

Аннотации лучше для typed-кода: явная документация, работает с наследованием, поддерживает свойства. Inspect проще для legacy-кода, но ломается с *args.

Аргументы дают полный контроль, но дают повторения - информация хранится как в самих аргументах, так и в слотах.

Для прода выбрала бы аннотации: они документируют типы, предсказуемы, соответствуют современным стандартам Python.  </font></b>

# `N-Singleton` strikes back

Реализуй метакласс `nsingleton`, который получает на вход n - максимальное количество экземпляров класса, которые могут существовать одновременно в ходе выполнения программы.

В случае, если у нас ещё нет n экземпляров класса, то он создаёт новый; если уже есть n экземпляров, то возвращает один из предыдущих, причём он итерируется по ним при последовательных вызовах(сначала вернёт первый экземпляр, потом второй и.т.д.)

In [171]:
from collections import deque

class nsingleton(type):
    _instances = {} 
    _counters = {} 
    _max_instances = {}

    def __new__(mcs, name, bases, namespace, n=1):
        namespace['_n'] = n
        return super().__new__(mcs, name, bases, namespace)

    def __init__(cls, name, bases, attrs, n=1):
        super().__init__(name, bases, attrs)
        n = getattr(cls, '_n', 1)
        cls._max_instances[cls] = n
        cls._instances[cls] = deque(maxlen=n)
        cls._counters[cls] = 0

    def __call__(cls, *args, **kwargs):
        max_n = cls._max_instances[cls]
        instances = cls._instances[cls]
        if len(instances) < max_n:
            instance = super().__call__(*args, **kwargs)
            instances.append(instance)
            return instance
        instance = instances[cls._counters[cls] % max_n]
        cls._counters[cls] = (cls._counters[cls] + 1) % max_n
        return instance


In [172]:
class LimitedHouse(metaclass=nsingleton, n=3):
    def __init__(self, name):
        self.name = name
        print(f"Создан дом: {name}")
    
    def __repr__(self):
        return f"House({self.name})"

h1 = LimitedHouse("1")
h2 = LimitedHouse("2")  
h3 = LimitedHouse("3")

print(h1)
print(h2)
print(h3)

h4 = LimitedHouse("4")
h5 = LimitedHouse("5")
h6 = LimitedHouse("6")
h7 = LimitedHouse("7")

print(h4, id(h4))
print(h5, id(h5)) 
print(h6, id(h6))
print(h7, id(h7))

print(h1 is h4)
print(h2 is h5)
print(h3 is h6)
print(h1 is h7)

Создан дом: 1
Создан дом: 2
Создан дом: 3
House(1)
House(2)
House(3)
House(1) 4495653392
House(2) 4496137168
House(3) 4495423312
House(1) 4495653392
True
True
True
True
